# Вийнятки в Python

## Ієрархія виключень
```python
BaseException
 +-- SystemExit
 +-- KeyboardInterrupt
 +-- GeneratorExit
 +-- Exception
      +-- StopIteration
      +-- AssertionError
      +-- AttributeError
      +-- LookupError
           +-- IndexError
           +-- KeyError
      +-- OSError
      +-- SystemError
      +-- TypeError
      +-- ValueError
```

In [1]:
print(1 / 0)

ZeroDivisionError: division by zero

In [ ]:
import sys


def wc(filename):
    count = 0
    with open(filename) as f:
        for line in f:
            count += 1

    return count


def process_file(filename):
    count = wc(filename)
    print(f'file {filename} has {count} lines')


def _main():
    process_file(sys.argv[1])


if __name__ == "__main__":
    _main()


In [3]:
class MyClass:
    pass

obj = MyClass()
obj.foo

AttributeError: 'MyClass' object has no attribute 'foo'

In [4]:
d = {'foo': 1}
d['bar']

KeyError: 'bar'

In [5]:
l = [1, 2]
l[10]

IndexError: list index out of range

In [6]:
int('asdf')

ValueError: invalid literal for int() with base 10: 'asdf'

In [7]:
1 + '10'

TypeError: unsupported operand type(s) for +: 'int' and 'str'

## Обробка виключень

In [8]:
try:
    1 / 0
except:
    print('Zero Division')

Zero Division


In [10]:
while True:
    try:
        raw = int(input('Enter a number:\t'))
        number = raw
        break
    except:
        print('Value error')

Enter a number:	bbb
Value error
Enter a number:	q
Value error
Enter a number:	100066600001


### Обробка кількох виключень

In [11]:
while True:
    try:
        number = int(input('Enter a number:\t'))
        break
    except ValueError:
        print('Value error')
    except KeyboardInterrupt:
        print('exit')

Enter a number:	
Value error
exit
Enter a number:	7


In [13]:
total = 100_000

while True:
    try:
        number = int(input('Enter a number:\t'))
        total /= number
        print(total)
        break
    except (ValueError, ZeroDivisionError):
        print('Value error')
    except KeyboardInterrupt:
        print('Interrupt by user')

Enter a number:	b
Value error
Interrupt by user
Enter a number:	0
Value error
Enter a number:	2
50000.0


In [15]:
# +-- LookupError
#     +-- IndexError
#     +-- KeyError

print(issubclass(IndexError, LookupError),
     issubclass(KeyError, LookupError))

True True


### наслідування

In [19]:
database = {
    'red': ['fox', 'flower'],
    'green': ['pease', 'M', 'python']
}

try:
    color = input('enter the color: \t')
    number = int(input('enter the order number: \t'))
    
    label = database[color][number]
    print('You choose:', label)
except LookupError:
    print('Diin\'t find the item')

enter the color: 	green
enter the order number: 	3
Diin't find the item


## Блок `finally`

In [22]:
f = open('/etc/hosts')

try:
    for line in f:
        print(line.rstrip('\n'))
        1/0
        
except OSError:
    print('Відмовлено в доступі')
finally:
    # Закриваємо файл у будь-якому випадку:
    f.close()

##


ZeroDivisionError: division by zero

## Доступ до об'єкта виключення

In [1]:
try:
    with open('/file/not/found') as f:
        content = f.read()
except OSError as err:
    print(err.errno, err.strerror)

2 No such file or directory


In [5]:
# атрибут ARGS
import os.path

filename = '/file/not/found'
try:
    if not os.path.exists(filename):
        raise ValueError('Файлу не існує', filename)
except ValueError as err:
    message, code = err.args[0], err.args[1]
    print(message, code)

Файлу не існує /file/not/found


In [6]:
# ДОСТУП ДО СТЕКУ ВИКЛІИКІВ
import traceback

try:
    with open('/file/not/found') as f:
        content = f.read()
except OSError as err:
    trace = traceback.print_exc()
    print(trace)

None


Traceback (most recent call last):
  File "<ipython-input-6-4431bbe3e8c5>", line 5, in <module>
    with open('/file/not/found') as f:
FileNotFoundError: [Errno 2] No such file or directory: '/file/not/found'


In [7]:
# Делегівання виключення вище
# з допомогою порожнього raise

try:
    raw = input('Enter y number: ')
    if not raw.isdigit():
        raise ValueError('bad number', raw)
except ValueError as err:
    print('некоректне значення', err)
    # делегування обробки виключення
    raise

Enter y number: -1
некоректне значення ('bad number', '-1')


ValueError: ('bad number', '-1')

### Виключення через raise form Exception

In [8]:
try:
    raw = input("введите число: ")
    
    if not raw.isdigit():
        raise ValueError("плохое число", raw)
except ValueError as err:
    print("ошибка:", err.args[0], err.args[1])
    
    raise TypeError("ошибка") from err

введите число: spam
ошибка: плохое число spam


TypeError: ошибка

## Інструкція `assert`

In [9]:
assert True
assert 1 == 0

AssertionError: 

In [10]:
assert 1 == 0, "1 не равен 0"

AssertionError: 1 не равен 0

## Інструкція `assert`, `flag -O`

In [11]:
def get_user_by_id(id):
    assert isinstance(id, int), "id должен быть целым числом"
    
    print("выполняем поиск")
 
if __name__ == "__main__":
    get_user_by_id("foo")

AssertionError: id должен быть целым числом

## Продуктивність винятків

In [12]:
%%timeit
my_dict = {"foo": 1}
for _ in range(1000):
    try:
        my_dict["bar"]
    except KeyError:
        pass

410 µs ± 11.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [13]:
%%timeit
my_dict = {"foo": 1}
for _ in range(1000):
    if "bar" in my_dict:
        _ = my_dict["bar"]

74.7 µs ± 1.31 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## Робота з особистими виключеннями.
### Бібліотека `requests`

In [15]:
import requests

response = requests.get("https://github-not-found.com")

ConnectionError: HTTPSConnectionPool(host='github-not-found.com', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x108360898>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [16]:
try:
    response = requests.get("https://github-not-found.com")
except requests.RequestException as err:
    print(err)

HTTPSConnectionPool(host='github-not-found.com', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x108806860>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))


In [18]:
# requests.__file__
import time

timeout = 0.2
for _ in range(5): 
    try:
        response = requests.get("https://github.com/not_found",
                                timeout=timeout)
        
        response.raise_for_status()
        break
    except requests.Timeout:
        print("попробуйте позже timeout:", timeout)
        timeout *= 2
        time.sleep(timeout)
    except requests.HTTPError as err:
        print(err.response.status_code)
        raise

404


HTTPError: 404 Client Error: Not Found for url: https://github.com/not_found